# Agents in Autogen

In [7]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [8]:
# Mock web search code
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "The Labrador Retriever or simply Labrador is a British breed of retreiver gun dog."

In [9]:
# Define Agent
agent = AssistantAgent(
    name='assistant',
    model_client=model_client,  # brain
    tools=[web_search],     # Tool to use
    system_message='Use Tools to solve tasks',
    description='An agent which uses tool to solve task'    # not requried
)

In [10]:
result = await agent.run(task='Find information about Labrador Retriever')
print(result.messages[-1].content)

The Labrador Retriever or simply Labrador is a British breed of retreiver gun dog.


In [11]:
result

TaskResult(messages=[TextMessage(id='ab65fab6-03a6-4f5a-a7b7-472c0573e7c2', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 1, 21, 17, 7, 6, 735367, tzinfo=datetime.timezone.utc), content='Find information about Labrador Retriever', type='TextMessage'), ToolCallRequestEvent(id='4b9b84bd-157e-4433-873f-268125caceb9', source='assistant', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=18), metadata={}, created_at=datetime.datetime(2026, 1, 21, 17, 7, 8, 708941, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_5urxbGV4Z3G37K09k5dTOsMY', arguments='{"query":"Labrador Retriever information"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='922fb5bf-7657-499b-803d-d1c341e06978', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 1, 21, 17, 7, 8, 710178, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The Labrador Retriever or simply Labrador

## Flow of messages explanation

#Notice how 'id' field is the same throughout messages
TaskResult(messages=
[
    #1st message - asking to find info about labrador retreiver
    1. TextMessage(id='7c53272a-cc73-4a05-bd40-965e257e6753', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 1, 21, 16, 59, 23, 385312, tzinfo=datetime.timezone.utc), content='Find information about Labrador Retriever', type='TextMessage'), 
    
    #Solve the question using the tool, so tool call request made, only sent "Labrador Retreiver"
    2. ToolCallRequestEvent(id='92f74a23-efbf-41e2-987c-d1cccc429207', source='assistant', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=17), metadata={}, created_at=datetime.datetime(2026, 1, 21, 16, 59, 24, 321669, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_owF0nsgNhE2F0Z9ylNsRQwdW', arguments='{"query":"Labrador Retriever"}', name='web_search')], type='ToolCallRequestEvent'), 
    
    #Call the tool and got the anwer
    3. ToolCallExecutionEvent(id='48ea95e6-585d-4f39-ab62-5d537343f65c', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 1, 21, 16, 59, 24, 323084, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The Labrador Retriever or simply Labrador is a British breed of retreiver gun dog.', name='web_search', call_id='call_owF0nsgNhE2F0Z9ylNsRQwdW', is_error=False)], type='ToolCallExecutionEvent'), 
    
    #This is our answer back
    4. ToolCallSummaryMessage(id='b7fc0583-d5e2-4f74-9b1d-c24c634cb3c1', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 1, 21, 16, 59, 24, 323143, tzinfo=datetime.timezone.utc), content='The Labrador Retriever or simply Labrador is a British breed of retreiver gun dog.', type='ToolCallSummaryMessage', tool_calls=[FunctionCall(id='call_owF0nsgNhE2F0Z9ylNsRQwdW', arguments='{"query":"Labrador Retriever"}', name='web_search')], results=[FunctionExecutionResult(content='The Labrador Retriever or simply Labrador is a British breed of retreiver gun dog.', name='web_search', call_id='call_owF0nsgNhE2F0Z9ylNsRQwdW', is_error=False)])], stop_reason=None)

## On_message() method

In [12]:
from autogen_core import CancellationToken

async def assistant_run()-> None:
    response = await agent.on_messages(
        messages= [TextMessage(content='Find information about Labrador Retreiver',source='User')],
        cancellation_token=CancellationToken()
    )

    print(response.inner_messages)
    print('\n\n\n\n')
    print(response.chat_message)

await assistant_run()

[]





id='199f4d55-9be2-4c0e-a160-0d26989a965c' source='assistant' models_usage=RequestUsage(prompt_tokens=117, completion_tokens=120) metadata={} created_at=datetime.datetime(2026, 1, 21, 17, 9, 12, 922847, tzinfo=datetime.timezone.utc) content='The Labrador Retriever, often simply called a Labrador, is a British breed of retriever gun dog. Known for its friendly nature, intelligence, and versatility, the Labrador Retriever is one of the most popular dog breeds in many countries. They are commonly used as service dogs, working dogs, and family pets due to their trainable and social nature. Labradors are also well-suited for activities such as hunting, agility sports, and search and rescue missions. They are known for their dense coat, otter tail, and strong build, typically coming in colors of black, yellow, and chocolate.' type='TextMessage'


[]





id='199f4d55-9be2-4c0e-a160-0d26989a965c' source='assistant' models_usage=RequestUsage(prompt_tokens=117, completion_tokens=120) metadata={} created_at=datetime.datetime(2026, 1, 21, 17, 9, 12, 922847, tzinfo=datetime.timezone.utc) content='The Labrador Retriever, often simply called a Labrador, is a British breed of retriever gun dog. Known for its friendly nature, intelligence, and versatility, the Labrador Retriever is one of the most popular dog breeds in many countries. They are commonly used as service dogs, working dogs, and family pets due to their trainable and social nature. Labradors are also well-suited for activities such as hunting, agility sports, and search and rescue missions. They are known for their dense coat, otter tail, and strong build, typically coming in colors of black, yellow, and chocolate.' type='TextMessage'

# Streaming Messages
## on_messages_stream()

In [14]:
from autogen_agentchat.ui import Console

async def assistant_run_stream() -> None:
    await Console(
        agent.on_messages_stream(
            messages= [TextMessage(content='Find information about Labrador Retreiver via the tool',source='User')],
            cancellation_token=CancellationToken()
        ),
        output_stats=True   #Enable stats printing
    )

await assistant_run_stream()

---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='call_8XssWIigqTD7o3Ao8W1J4ngC', arguments='{"query":"Labrador Retriever breed information"}', name='web_search')]
[Prompt tokens: 256, Completion tokens: 19]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='The Labrador Retriever or simply Labrador is a British breed of retreiver gun dog.', name='web_search', call_id='call_8XssWIigqTD7o3Ao8W1J4ngC', is_error=False)]
---------- assistant ----------
The Labrador Retriever or simply Labrador is a British breed of retreiver gun dog.
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 256
Total completion tokens: 19
Duration: 2.63 seconds


In [ ]:
# Prove that it remembers previous context (from new message, didn't have to maintain history ourself)
from autogen_agentchat.ui import Console

async def assistant_run_stream() -> None:
    await Console(
        agent.on_messages_stream(
            messages= [TextMessage(content='What was the last question I asked?',source='User')],
            cancellation_token=CancellationToken()
        ),
        output_stats=True   #Enable stats printing
    )

await assistant_run_stream()

---------- assistant ----------
The last question you asked was: "Find information about Labrador Retreiver via the tool"
[Prompt tokens: 314, Completion tokens: 20]
---------- Summary ----------
Number of inner messages: 0
Total prompt tokens: 314
Total completion tokens: 20
Duration: 4.12 seconds
